In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import resnet34
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [3]:
# TODO: FILL IN PARAMETERS
# Model Parameter
model = "resnet34"
model_short = "resnet"
MODEL_MODULE = resnet34
# Dataset parameter
dataset = "cifar10"
dataset_short = "cifar"
DATASET_ENUM = DatasetClassN.Cifar

## Attack model training parameters
LEARNING_R = 0.001
EPOCHS = 3
MULTI_WORKERS_N = 4

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [4]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [5]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [6]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH, standardize=True)

Attack dataset was already established previosly, loading dataset from "pickle/cifar10/resnet34/attack_train.p".


In [7]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 2.7610,  1.5855,  1.0994]],  [[ 3.6989,  0.6983, -0.1858]],  [[10.9323,  0.6241, -1.9186]],  [[ 8.2997,  1.8134, -1.3457]],  [[ 6.8367,  1.1840,  0.3727]],  [[ 6.0857,  1.5329,  0.7576]],  [[ 6.4071,  1.6038, -0.8482]],  [[ 7.3147,  6.2227,  1.0931]],  [[ 4.8549,  1.4512, -0.4886]],  [[ 4.7426,  4.6196,  2.3844]],  [[10.3158,  2.3646,  0.1576]],  [[ 4.8396,  1.6346,  1.3971]],  [[ 4.9852,  1.6474, -0.4738]],  [[14.1636, -1.2570, -2.5510]],  [[ 7.3456,  3.6341,  1.0219]],  [[ 6.1835,  1.0170,  0.5863]],  [[ 6.3865,  3.7981, -0.3039]],  [[ 4.5035,  4.1631,  0.3921]],  [[ 5.8886,  4.4171,  2.7559]],  [[10.2302, -1.6795, -2.3491]],  [[ 5.6295,  3.4255,  1.4393]],  [[ 4.5133,  1.8663,  1.8542]],  [[11.4608,  2.8926, -0.6610]],  [[ 5.2633,  4.3472, -0.9264]],  [[ 7.8379,  1.6665,  1.3011]],  [[10.1063,  1.5136,  1.4417]],  [[ 4.1329,  2.1567,  0.6392]],  [[10.1602,  3.8488,  1.4402]],  [[ 8.3354,  1.8722, -1.1975]],  [[ 3.6238,  2.2319,  0.5668]],  [[ 9.0621,  0.2615,  0.1875]], 

### Attack Model

Simple Model

In [8]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [9]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
       

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [10]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [11]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, DEVICE, test_dataset=False, standardize=True)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.3373, 0.3765, 0.3843,  ..., 0.2078, 0.2118, 0.2235], [0.4392, 0.5412, 0.5059,  ..., 0.2510, 0.2549, 0.2235], [0.4275, 0.4510, 0.4471,  ..., 0.2627, 0.2549, 0.2314], ..., [0.5373, 0.5647, 0.5529,  ..., 0.3490, 0.2745, 0.1765], [0.4157, 0.4353, 0.4549,  ..., 0.1608, 0.0941, 0.0745], [0.3882, 0.4000, 0.3961,  ..., 0.0627, 0.0627, 0.0627]],[[0.3765, 0.4314, 0.4314,  ..., 0.3412, 0.3333, 0.3490], [0.5529, 0.6784, 0.6275,  ..., 0.4039, 0.4157, 0.3804], [0.5059, 0.5569, 0.5765,  ..., 0.4235, 0.4118, 0.3843], ..., [0.6627, 0.7059, 0.7059,  ..., 0.3843, 0.2902, 0.1843], [0.4824, 0.5137, 0.5373,  ..., 0.1843, 0.1098, 0.0824], [0.4431, 0.4627, 0.4627,  ..., 0.0745, 0.0745, 0.0745]],[[0.1882, 0.2157, 0.2549,  ..., 0.4980, 0.4941, 0.5176], [0.1882, 0.2941, 0.2745,  ..., 0.5647, 0.5882, 0.5490], [0.3804, 0.2941, 0.2353,  ..., 0.5843, 0.5804, 0.5569], ..., [0.7765, 0.8314,

Attack Evaluation DL Sample

In [12]:
eval_sample_iter = iter(attack_eval_post_loader)
print (str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 8.2969e+00,  4.9705e-01,  3.0791e-01]],  [[ 1.0034e+01, -5.1437e-01, -6.1488e-01]],  [[ 9.3802e+00,  4.1555e-01, -4.3127e-01]],  [[ 1.0471e+01, -8.2182e-02, -4.2400e-01]],  [[ 9.2651e+00,  4.1612e-02,  2.7437e-02]],  [[ 9.0344e+00,  5.6719e-01, -8.2624e-01]],  [[ 7.8524e+00,  4.1818e-01, -1.2401e-01]],  [[ 9.0963e+00,  4.6028e-01, -6.6548e-02]],  [[ 9.3732e+00,  8.5999e-02, -3.7518e-01]],  [[ 8.2181e+00,  6.9449e-01,  4.7225e-01]],  [[ 9.8387e+00,  5.3789e-01,  1.6403e-01]],  [[ 1.1495e+01, -4.4880e-01, -5.2808e-01]],  [[ 8.1442e+00,  6.3936e-02, -5.5746e-01]],  [[ 8.8131e+00, -5.3464e-02, -5.1176e-01]],  [[ 8.9089e+00, -2.0440e-01, -2.7369e-01]],  [[ 8.3238e+00,  3.2423e-01, -2.2219e-01]],  [[ 9.1166e+00,  6.2863e-01, -3.0783e-01]],  [[ 1.0020e+01,  3.4747e-01, -5.6266e-01]],  [[ 8.7493e+00,  2.3795e-01,  1.7270e-01]],  [[ 9.6765e+00, -3.6445e-02, -4.0399e-01]],  [[ 9.6246e+00,  8.5195e-01, -1.9971e-01]],  [[ 9.2846e+00,  1.2553e-01, -2.5514e-01]],  [[ 1.0460e+01,  1.0478e

### Train Attack model

In [13]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Attack model state dictionary already available, loading into input model from attack_models/attack_resnet_cifar.pth!


### Evaluation

In [14]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 28.56
